In [0]:
#cloning the git repo for the dataset
!git clone https://github.com/techytushar/Sign-Language-Recognizer

In [0]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, MaxPool2D, Conv2D, Dropout
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import os
import cv2

In [0]:
#X is the training set, Y is the train lables
X = np.zeros((1,100,100,1), dtype=float)
Y = np.zeros((1,10),dtype=float)

In [0]:
# reading images, converting to grayscale and adding them to array
path = "./Sign-Language-Recognizer/Dataset"
for i in os.listdir(path):
    print(f"Reading images for number {i}")
    for j in os.listdir(f'{path}/{i}'):
        img = cv2.imread(f'{path}/{i}/{j}')
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        try:
            assert np.prod(gray.shape) == 10000
        except AssertionError as e:
            gray = cv2.resize(gray, (100,100))
        gray = gray/255
        gray = gray.reshape(1,100,100,1)
        X = np.vstack((X,gray))
        y = np.zeros((1,10))
        y[0,int(i)] = 1
        Y = np.vstack((Y,y))

In [0]:
# removing the zeros array and shuffeling the data
X = X[1:,:,:,:]
Y = Y[1:,:]
X,Y = shuffle(X,Y)
print(X.shape, Y.shape)

In [0]:
#visualizing the images
image_number=1999
plt.imshow(X[image_number,:,:].reshape(100,100))
plt.show()
print(Y[image_number])

In [0]:
#splitting the data into train and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.1)
print(X_train.shape, X_test.shape)

In [0]:
#defining the model
model = Sequential()
model.add(Conv2D(32, kernel_size=3, padding="same",  input_shape=(100, 100, 1), activation="relu"))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Conv2D(64, kernel_size=3, padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(units=512, activation="relu"))
model.add(Dropout(0.4))
model.add(Dense(units=128, activation="relu"))
model.add(Dropout(0.25))
model.add(Dense(units=10, activation="softmax"))

In [0]:
#training the model
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=['accuracy'])
model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=47, batch_size=128)

In [0]:
#saving the model
model.save("hand_sign_keras.h5")

In [0]:
#saving the model from colab to google drive (code from StackOverflow)
from google.colab import auth
from googleapiclient.http import MediaFileUpload
from googleapiclient.discovery import build

auth.authenticate_user()

In [0]:
drive_service = build('drive', 'v3')

def save_file_to_drive(name, path):
  file_metadata = {
    'name': name,
    'mimeType': 'application/octet-stream'
   }
  media = MediaFileUpload(path, 
                  mimetype='application/octet-stream',
                  resumable=True)

  created = drive_service.files().create(body=file_metadata,
                                 media_body=media,
                                 fields='id').execute()

  print('File ID: {}'.format(created.get('id')))

  return created

In [0]:
save_file_to_drive("model_final.h5", "hand_sign_keras.h5")